In [24]:
import numpy as np
import pandas as pd
from arch import arch_model

np.random.seed(42)   # 🔒 khóa kết quả


In [2]:
df = pd.read_excel(
    r"C:\Users\Admin\Downloads\Data.xlsx",
    sheet_name="WORK"
)
df.columns = df.columns.str.strip().str.lower()

In [3]:
df["ngày"] = pd.to_datetime(df["ngày"])
df = df.sort_values("ngày")
df = df.set_index("ngày")

In [5]:
df["ler"]     = np.log(df["er"])
df["lbtc"]    = np.log(df["btcu"])
df["leth"]    = np.log(df["ethu"])
df["lgold"]   = np.log(df["goldu"])
df["lbrent"]  = np.log(df["brentu"])
df["lvni"]    = np.log(df["vni"])

In [6]:
(df[["er","btcu","ethu","goldu","brentu","vni"]] <= 0).sum()

er        0
btcu      0
ethu      0
goldu     0
brentu    0
vni       0
dtype: int64

In [25]:
returns = pd.DataFrame({
    'rer':    df['ler'].diff(),
    'rbtc':   df['lbtc'].diff(),
    'reth':   df['leth'].diff(),
    'rgold':  df['lgold'].diff(),
    'rbrent': df['lbrent'].diff(),
    'rvni':   df['lvni'].diff()
}).dropna()


In [26]:
periods = {
    'Full sample': (returns.index.min(), returns.index.max()),
    'Pre-COVID': ('2015-01-01', '2019-12-31'),
    'COVID': ('2020-01-01', '2021-12-31'),
    'Ukraine': ('2022-01-01', '2023-12-31')
}


In [8]:
df = df.dropna()

In [31]:
def fit_garch(series):
    model = arch_model(
        series,
        mean='Constant',
        vol='Garch',
        p=1, q=1,
        dist='normal'
    )
    res = model.fit(disp='off')
    return res.conditional_volatility, res.std_resid



In [28]:
def dcc_garch(z1, z2, a=0.05, b=0.93):
    Z = np.column_stack([z1, z2])
    Qbar = np.cov(Z.T)
    Qt = Qbar.copy()
    R_list = []

    for t in range(len(Z)):
        Qt = (1 - a - b) * Qbar \
             + a * np.outer(Z[t-1], Z[t-1]) \
             + b * Qt
        Dinv = np.diag(1 / np.sqrt(np.diag(Qt)))
        Rt = Dinv @ Qt @ Dinv
        R_list.append(Rt)

    return R_list


In [29]:
def hedge_ratio(R_list, vol_a, vol_f):
    beta = []
    for t, Rt in enumerate(R_list):
        cov = Rt[0,1] * vol_a.iloc[t] * vol_f.iloc[t]
        beta.append(cov / (vol_f.iloc[t]**2))
    return pd.Series(beta, index=vol_a.index)

def hedge_effectiveness(r_a, r_f, beta):
    hedged = r_a - beta * r_f
    return 1 - np.var(hedged) / np.var(r_a)


In [34]:
results = []

for pname, (start, end) in periods.items():

    data_p = returns.loc[start:end]

    vol_btc, z_btc = fit_garch(data_p['rbtc'])
    vol_er,  z_er  = fit_garch(data_p['rer'])

    R = dcc_garch(z_btc, z_er)

    beta = hedge_ratio(R, vol_btc, vol_er)
    HE = hedge_effectiveness(data_p['rbtc'], data_p['rer'], beta)

    results.append({
        'Asset': 'BTC_ER',
        'Period': pname,
        'HE': HE,
        'Beta_mean': beta.mean(),
        'Beta_std': beta.std()
    })


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.00122. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 1.406e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Admin\App

In [35]:
results_df = pd.DataFrame(results)
results_df


,Asset,Period,HE,Beta_mean,Beta_std
0,BTC_ER,Full sample,-0.036722,-1.147463,6.172917
1,BTC_ER,Pre-COVID,-0.022204,3.959354,7.774201
2,BTC_ER,COVID,-0.072130,-7.662551,12.076082
3,BTC_ER,Ukraine,-0.026645,-0.128258,3.340409


In [36]:
def hedge_ratio(R_list, vol_gold, vol_er):
    beta = []
    for t, Rt in enumerate(R_list):
        cov = Rt[0,1] * vol_gold.iloc[t] * vol_er.iloc[t]
        beta.append(cov / (vol_er.iloc[t]**2))
    return pd.Series(beta, index=vol_gold.index)

def hedge_effectiveness(r_gold, r_er, beta):
    hedged = r_gold - beta * r_er
    return 1 - np.var(hedged) / np.var(r_gold)


In [39]:
results = []

for pname, (start, end) in periods.items():

    data_p = returns.loc[start:end]

    # GARCH
    vol_gold, z_gold = fit_garch(data_p['rgold'])
    vol_er,   z_er   = fit_garch(data_p['rer'])

    # DCC
    R = dcc_garch(z_gold, z_er)

    # Hedge
    beta = hedge_ratio(R, vol_gold, vol_er)
    HE = hedge_effectiveness(data_p['rgold'], data_p['rer'], beta)

    results.append({
        'Asset': 'GOLD_ER',
        'Period': pname,
        'HE': HE,
        'Beta_mean': beta.mean(),
        'Beta_std': beta.std()
    })


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.844e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 1.406e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Admin\

In [40]:
results_gold_er = pd.DataFrame(results)
results_gold_er


,Asset,Period,HE,Beta_mean,Beta_std
0,GOLD_ER,Full sample,-0.022447,-0.086824,1.287855
1,GOLD_ER,Pre-COVID,-0.044220,-0.173088,1.379370
2,GOLD_ER,COVID,-0.033653,-0.495926,2.286339
3,GOLD_ER,Ukraine,-0.018474,-0.012560,0.823293


In [ ]:
def fit_garch(series):
    model = arch_model(series*100, p=1, q=1, mean="Zero")
    return model.fit(disp="off")


In [ ]:
def fit_dcc(regime_df):
    # ===== Univariate GARCH =====
    res_btc = fit_garch(regime_df["rbtc"])
    res_er  = fit_garch(regime_df["rer"])

    z1 = res_btc.std_resid
    z2 = res_er.std_resid

    Z = np.column_stack([z1, z2])

    # ===== DCC estimation =====
    opt = minimize(
        dcc_loglik,
        x0=[0.05, 0.90],
        args=(Z,),
        bounds=[(0,1),(0,1)],
        constraints=[{"type":"ineq", "fun":lambda x: 1-x[0]-x[1]}]
    )

    return {
        "dcc_a": opt.x[0],
        "dcc_b": opt.x[1],
        "loglik": -opt.fun,
        "z": Z,
        "sigma1": res_btc.conditional_volatility,
        "sigma2": res_er.conditional_volatility
    }

In [11]:
pip install pandas numpy scipy statsmodels arch


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
returns = pd.DataFrame({
    'rer':    df['ler'].diff(),
    'rbtc':   df['lbtc'].diff(),
    'reth':   df['leth'].diff(),
    'rgold':  df['lgold'].diff(),
    'rbrent': df['lbrent'].diff(),
    'rvni':   df['lvni'].diff()
}).dropna()


In [13]:
from arch import arch_model

def fit_garch(series):
    model = arch_model(series, vol='Garch', p=1, q=1, dist='normal')
    res = model.fit(disp='off')
    return res.conditional_volatility, res.std_resid

btc_vol, btc_std = fit_garch(returns['rbtc'])
er_vol,  er_std  = fit_garch(returns['rer'])


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.00122. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 1.406e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Admin\App

In [14]:
T = len(btc_std)
z = np.column_stack([btc_std, er_std])

Qbar = np.cov(z.T)
a, b = 0.05, 0.93   # có thể MLE sau
Qt = Qbar.copy()

R_list = []

for t in range(T):
    Qt = (1-a-b)*Qbar + a*np.outer(z[t-1], z[t-1]) + b*Qt
    Dinv = np.diag(1/np.sqrt(np.diag(Qt)))
    Rt = Dinv @ Qt @ Dinv
    R_list.append(Rt)


In [15]:
print(R_list)

[array([[ 1.        , -0.01668689],
       [-0.01668689,  1.        ]]), array([[1.        , 0.00430065],
       [0.00430065, 1.        ]]), array([[1.        , 0.00379101],
       [0.00379101, 1.        ]]), array([[ 1.        , -0.02178933],
       [-0.02178933,  1.        ]]), array([[ 1.        , -0.02310403],
       [-0.02310403,  1.        ]]), array([[ 1.        , -0.01496623],
       [-0.01496623,  1.        ]]), array([[1.        , 0.06821895],
       [0.06821895, 1.        ]]), array([[1.        , 0.03937628],
       [0.03937628, 1.        ]]), array([[1.        , 0.01086213],
       [0.01086213, 1.        ]]), array([[1.        , 0.08152102],
       [0.08152102, 1.        ]]), array([[1.        , 0.10862526],
       [0.10862526, 1.        ]]), array([[1.        , 0.10258261],
       [0.10258261, 1.        ]]), array([[1.        , 0.10300795],
       [0.10300795, 1.        ]]), array([[1.        , 0.10009936],
       [0.10009936, 1.        ]]), array([[1.        , 0.07592376]

In [16]:
beta_t = []

for t, Rt in enumerate(R_list):
    cov = Rt[0,1] * btc_vol.iloc[t] * er_vol.iloc[t]
    var = er_vol.iloc[t]**2
    beta_t.append(cov / var)

beta_t = pd.Series(beta_t, index=returns.index)


In [17]:
print(beta_t)

ngày
2018-01-03   -3.432144
2018-01-04    0.762649
2018-01-05    0.607692
2018-01-06   -3.627547
2018-01-07   -3.533464
                ...   
2025-09-26   -0.075595
2025-09-27    0.028725
2025-09-28    0.024634
2025-09-29   -0.207708
2025-09-30    0.963288
Length: 2828, dtype: float64


In [18]:
unhedged = returns['rbtc']
hedged = returns['rbtc'] - beta_t * returns['rer']

HE = 1 - np.var(hedged) / np.var(unhedged)
print("Hedge Effectiveness:", HE)


Hedge Effectiveness: -0.03672228643471054


In [20]:
results = []

pairs = {
    'BTC–ER': ('rbtc', 'rer'),
    'GOLD–ER': ('rgold', 'rer'),
    'ETH–ER': ('reth', 'rer')
}
